In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/var/folders/1x/gfq_5w6527q1mtcmjs8xlyxr0000gn/T/ipykernel_37819/3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Lab. Natural Lenguage Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [5]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("data/kg_train.csv",encoding='latin-1')

# Reduce the training set to speead up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


### Let's divide the training and test set into two partitions

In [6]:
# Your code
X = data['text']
y = data['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training set size:", X_train.shape[0])
print("Test set size:", X_test.shape[0])

Training set size: 800
Test set size: 200


## Data Preprocessing

In [7]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
import string
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each']


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gerardroca/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/gerardroca/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/gerardroca/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [8]:
# Your code
import re
from bs4 import BeautifulSoup

def clean_html(html: str) -> str:
    html_no_js = re.sub(r'<script.*?>.*?</script>', '', html, flags=re.DOTALL | re.IGNORECASE)    
    html_no_css = re.sub(r'<style.*?>.*?</style>', '', html_no_js, flags=re.DOTALL | re.IGNORECASE)
    html_no_comments = re.sub(r'<!--.*?-->', '', html_no_css, flags=re.DOTALL | re.IGNORECASE)
    soup = BeautifulSoup(html_no_comments, "html.parser")
    html_no_tags = soup.get_text(separator=" ")
    text_only = re.sub(r'<[^>]+>', ' ', html_no_tags)
    return text_only

X_train = X_train.apply(clean_html)
print(X_train.head())



/var/folders/1x/gfq_5w6527q1mtcmjs8xlyxr0000gn/T/ipykernel_37819/1733798072.py:9: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(html_no_comments, "html.parser")
/var/folders/1x/gfq_5w6527q1mtcmjs8xlyxr0000gn/T/ipykernel_37819/1733798072.py:9: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  soup = BeautifulSoup(html_no_comments, "html.parser")


29     ----------- REGARDS, MR NELSON SMITH.KINDLY RE...
535    I have not been able to reach oscar this am. W...
695    ; Huma Abedin B6I'm checking with Pat on the 5...
557    I can have it announced here on Monday - can't...
836        BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 S...
Name: text, dtype: object


- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [10]:
# Your code
def clean_text(text:str) -> str:
    text_no_spc_char = re.sub(r'[^A-Za-z\s]', '', text)
    text_no_numbers = re.sub(r'\d+', '', text_no_spc_char)
    text_no_single_char = re.sub(r'^[a-zA-Z]\s+', '', text_no_numbers)
    text_no_single_char_start = re.sub(r'^[a-zA-Z]\s+', '', text_no_single_char)
    text_no_multiple_spaces = re.sub(r'\s+', ' ', text_no_single_char_start).strip()
    text_removed_prefixed_b = text_no_multiple_spaces.lstrip('b')
    text_lowercase = text_removed_prefixed_b.lower()
    return text_lowercase

data['cleaned_html'] = data['text'].apply(clean_html)
data['cleaned_text'] = data['cleaned_html'].apply(clean_text)
print(data['cleaned_text'].head())


/var/folders/1x/gfq_5w6527q1mtcmjs8xlyxr0000gn/T/ipykernel_37819/1733798072.py:9: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(html_no_comments, "html.parser")
/var/folders/1x/gfq_5w6527q1mtcmjs8xlyxr0000gn/T/ipykernel_37819/1733798072.py:9: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  soup = BeautifulSoup(html_no_comments, "html.parser")


0    dear sir strictly a private business proposal ...
1                                              will do
2    noracheryl has emailed dozens of memos about h...
3    dear sirfmadamc i know that this proposal migh...
4                                                  fyi
Name: cleaned_text, dtype: object


## Now let's work on removing stopwords
Remove the stopwords.

In [17]:
# Your code
def remove_stopwords(text:str) -> str:
    stop_words = set(stopwords.words('english'))
    text_no_stopwords = ' '.join(word for word in text.split() if word not in stop_words)
    return text_no_stopwords

data['no_stopwords_text'] = data['cleaned_text'].apply(remove_stopwords)

print(data['no_stopwords_text'].head())

0    dear sir strictly private business proposal mi...
1                                                     
2    noracheryl emailed dozens memos haiti weekend ...
3    dear sirfmadamc know proposal might surprise e...
4                                                  fyi
Name: no_stopwords_text, dtype: object


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [18]:
# Your code

def lemmatize_text(text: str) -> str:
    lemmatizer = WordNetLemmatizer()
    lemmatized_text = ' '.join([lemmatizer.lemmatize(word) for word in text.split()])
    return lemmatized_text

data['final_cleaned_text'] = data['no_stopwords_text'].apply(lemmatize_text)

print(data['final_cleaned_text'].head())


0    dear sir strictly private business proposal mi...
1                                                     
2    noracheryl emailed dozen memo haiti weekend pl...
3    dear sirfmadamc know proposal might surprise e...
4                                                  fyi
Name: final_cleaned_text, dtype: object


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [21]:
# Your code
from collections import Counter
import string
# Separate ham and spam messages
ham_messages = data[data['label'] == 0]['final_cleaned_text']
spam_messages = data[data['label'] == 1]['final_cleaned_text']

def get_top_n_words(text_series, n=10):
    all_words = ' '.join(text_series).split()
    word_counts = Counter(all_words)
    return word_counts.most_common(n)

top_ham_words = get_top_n_words(ham_messages)
top_spam_words = get_top_n_words(spam_messages)

print("Top 10 words in ham messages:", top_ham_words)
print("Top 10 words in spam messages:", top_spam_words)

Top 10 words in ham messages: [('u', 135), ('pm', 115), ('would', 106), ('state', 103), ('president', 94), ('call', 91), ('time', 84), ('percent', 77), ('secretary', 76), ('work', 73)]
Top 10 words in spam messages: [('money', 926), ('account', 801), ('bank', 748), ('fund', 709), ('u', 566), ('business', 476), ('transaction', 426), ('country', 414), ('transfer', 399), ('million', 389)]


## Extra features

In [24]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_symbol_list = "|".join(["euro","dollar","pound","€","$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data['money_mark'] = data['final_cleaned_text'].str.contains(money_symbol_list, case=False, regex=True) * 1
data['suspicious_words'] = data['final_cleaned_text'].str.contains(suspicious_words, case=False, regex=True) * 1
data['text_len'] = data['final_cleaned_text'].apply(lambda x: len(x))

data.head()

,text,label,cleaned_html,cleaned_text,final_cleaned_text,no_stopwords_text,money_mark,suspicious_words,text_len
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",dear sir strictly a private business proposal ...,dear sir strictly private business proposal mi...,dear sir strictly private business proposal mi...,1,1,1504
1,Will do.,0,Will do.,will do,,,1,0,0
2,Nora--Cheryl has emailed dozens of memos about...,0,Nora--Cheryl has emailed dozens of memos about...,noracheryl has emailed dozens of memos about h...,noracheryl emailed dozen memo haiti weekend pl...,noracheryl emailed dozens memos haiti weekend ...,1,0,110
3,Dear Sir=2FMadam=2C I know that this proposal ...,1,Dear Sir=2FMadam=2C I know that this proposal ...,dear sirfmadamc i know that this proposal migh...,dear sirfmadamc know proposal might surprise e...,dear sirfmadamc know proposal might surprise e...,1,1,1382
4,fyi,0,fyi,fyi,fyi,fyi,1,0,3


## How would work the Bag of Words with Count Vectorizer concept?

In [ ]:
# Your code

## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [25]:
# Your code
vectorizer = TfidfVectorizer()

X_tfidf = vectorizer.fit_transform(data['final_cleaned_text'])
print("Shape of the TF-IDF vectorized dataset:", X_tfidf.shape)

X_tfidf_df = pd.DataFrame(X_tfidf.toarray(), columns=vectorizer.get_feature_names_out())
print(X_tfidf_df.head())

Shape of the TF-IDF vectorized dataset: (1000, 19006)
   aac  aaclocated  aae  aag  aaronovitchon  abacha  abachabefore  abachac  \
0  0.0         0.0  0.0  0.0            0.0     0.0           0.0      0.0   
1  0.0         0.0  0.0  0.0            0.0     0.0           0.0      0.0   
2  0.0         0.0  0.0  0.0            0.0     0.0           0.0      0.0   
3  0.0         0.0  0.0  0.0            0.0     0.0           0.0      0.0   
4  0.0         0.0  0.0  0.0            0.0     0.0           0.0      0.0   

   abachace  abachaco  ...  zongo  zongothe  zuhair  \
0       0.0       0.0  ...    0.0       0.0     0.0   
1       0.0       0.0  ...    0.0       0.0     0.0   
2       0.0       0.0  ...    0.0       0.0     0.0   
3       0.0       0.0  ...    0.0       0.0     0.0   
4       0.0       0.0  ...    0.0       0.0     0.0   

   zulatoebikozulatonetscapeenet  zulatoffffffffffffffffff      zuma    zumac  \
0                            0.0                       0.0  0.000

## And the Train a Classifier?

In [27]:
# Your code
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

X_train_tfidf, X_test_tfidf, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

classifier = MultinomialNB()

classifier.fit(X_train_tfidf, y_train)

y_pred = classifier.predict(X_test_tfidf)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

print(classification_report(y_test, y_pred))

Accuracy: 0.945
              precision    recall  f1-score   support

           0       0.99      0.92      0.95       125
           1       0.88      0.99      0.93        75

    accuracy                           0.94       200
   macro avg       0.94      0.95      0.94       200
weighted avg       0.95      0.94      0.95       200



### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).

In [ ]:
# Your code